# Biostat 257 Homework 4

**Due May 29 @ 11:59PM**

In [1]:
versioninfo()

Julia Version 1.4.0
Commit b8e9a9ecc6 (2020-03-21 16:36 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin18.6.0)
  CPU: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-8.0.1 (ORCJIT, skylake)


In this assignment, we continue with the linear mixed effects model (LMM) considered in HW2
$$
    \mathbf{Y}_i = \mathbf{X}_i \boldsymbol{\beta} + \mathbf{Z}_i \boldsymbol{\gamma} + \boldsymbol{\epsilon}_i, \quad i=1,\ldots,n,
$$
where   
- $\mathbf{Y}_i \in \mathbb{R}^{n_i}$ is the response vector of $i$-th individual,  
- $\mathbf{X}_i \in \mathbb{R}^{n_i \times p}$ is the fixed effects predictor matrix of $i$-th individual,  
- $\mathbf{Z}_i \in \mathbb{R}^{n_i \times q}$ is the random effects predictor matrix of $i$-th individual,  
- $\boldsymbol{\epsilon}_i \in \mathbb{R}^{n_i}$ are multivariate normal $N(\mathbf{0}_{n_i},\sigma^2 \mathbf{I}_{n_i})$,  
- $\boldsymbol{\beta} \in \mathbb{R}^p$ are fixed effects, and  
- $\boldsymbol{\gamma} \in \mathbb{R}^q$ are random effects assumed to be $N(\mathbf{0}_q, \boldsymbol{\Sigma}_{q \times q}$) independent of $\boldsymbol{\epsilon}_i$.

The log-likelihood of the $i$-th datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$ is 
$$
    \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) = - \frac{n_i}{2} \log (2\pi) - \frac{1}{2} \log \det \boldsymbol{\Omega}_i - \frac{1}{2} (\mathbf{y} - \mathbf{X}_i \boldsymbol{\beta})^T \boldsymbol{\Omega}_i^{-1} (\mathbf{y} - \mathbf{X}_i \boldsymbol{\beta}),
$$
where
$$
    \boldsymbol{\Omega}_i = \sigma^2 \mathbf{I}_{n_i} + \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T = \sigma^2 \mathbf{I}_{n_i} + \mathbf{Z}_i \mathbf{L} \mathbf{L}^T \mathbf{Z}_i^T,
$$
Because the variance component parameter $\boldsymbol{\Sigma}$ has to be positive semidefinite. We use its Cholesky factor $\mathbf{L}$ as optimization variable. 

Given $m$ independent data points $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$, $i=1,\ldots,m$, we seek the maximum likelihood estimate (MLE) by maximizing the log-likelihood
$$
\ell(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2) = \sum_{i=1}^m \ell_i(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2).
$$
In this assignment, we use the nonlinear programming (NLP) approach for optimization. In HW5 and HW6, we will derive an EM (expectation-maximization) algorithm and an MM (minorization-maximization) algorithm for the same problem. 

In [2]:
# load necessary packages; make sure install them first
using BenchmarkTools, CSV, DataFrames, DelimitedFiles, Distributions
using Ipopt, LinearAlgebra, MathProgBase, MixedModels, NLopt
using Random, RCall, Revise

## Q1. (Optional, 30 bonus pts) Derivatives

NLP optimization solvers expect users to provide at least a function for evaluating objective value. If users can provide further information such as gradient and Hessian, the NLP solvers will be more stable and converge faster. Automatic differentiation tools are becoming more powerful but cannot apply to all problems yet.

1. Prove that the gradient of $\ell_i$ is
\begin{eqnarray*}
\nabla_{\boldsymbol{\beta}} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) &=& \mathbf{X}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i, \\
\nabla_{\sigma^2} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) &=& - \frac{1}{2} \operatorname{tr} (\boldsymbol{\Omega}_i^{-1}) + \frac{1}{2} \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-2} \mathbf{r}_i, \\
\frac{\partial}{\partial \mathbf{L}} \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma^2) &=& - \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L} + \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{r}_i \mathbf{r}_i^T \boldsymbol{\Omega}_i^{-1} \mathbf{Z}_i \mathbf{L},
\end{eqnarray*}
where $\mathbf{r}_i = \mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta}$. 

2. Derive the observed information matrix and the expected (Fisher) information matrix.

If you need a refresher on multivariate calculus, my [Biostat 216 lecture notes](https://ucla-biostat216-2019fall.github.io/slides/16-matrixcalc/16-matrixcalc.html) may be helpful.

1. $∇_β$ is decently straightforward.
$$
\begin{align*}
∇_β&\Big\{-\frac{1}{2}[y_i'\Omega_i^{-1}y_i-2\beta'X_i\Omega_i^{-1}y_i+\beta'X_i'\Omega_i^{-1}X_i\beta]\Big\}\\
&=X_i'\Omega^{-1}y_i-X_i'\Omega_i^{-1}X_i\beta\\
&=X_i'[\Omega_i^{-1}y_i-\Omega_i^{-1}X_i\beta]\\
&=X_i'\Omega_i^{-1}r_i
\end{align*}
$$

2. To find $∇_{\sigma_0^2}$, I'll break into two steps. The first is
$$
\begin{align*}
\nabla_{\sigma_0^2}\Big\{-\frac{1}{2}\log\det(\sigma_0^2I+Z_i\Sigma Z_i')\Big\}=-\frac{1}{2}tr(\Omega^{-1}_i)\text{ by the matrix cookbook.}
\end{align*}
$$
The second is
$$
\begin{align*}
\nabla\Big\{-\frac{1}{2}r_i'\Omega_i^{-1}r_i\Big\}&=-\frac{1}{2}\nabla tr(r_i'\Omega_i^{-1}r_i)=-\frac{1}{2}tr(\nabla r_i'\Omega_i^{-1}r_i)\\
&=\frac{1}{2}tr(\Omega_i^{-1}r_ir_i'\Omega_i^{-1})\text{ by rule (61) in the matrix cookbook.}\\
&=\frac{1}{2}r_i'\Omega_i^{-2}r_i
\end{align*}
$$
Combining them I get
$$
-\frac{1}{2}tr(\Omega^{-1}_i)+\frac{1}{2}r_i'\Omega_i^{-2}r_i
$$
3. To find $\partial L$, I'll again break into two parts corresponding to the pieces in the log-likelihood. Taking the partial of the determinant part gives
$$
\begin{align*}
-\frac{1}{2}\Omega_i^{-1}\frac{\partial}{\partial L}(\sigma_0^2 I+Z_iLL'Z_i')
\end{align*}
$$
where 
$$
\frac{\partial}{\partial L}(\sigma_0^2 I+Z_iLL'Z_i')=Z_iL\otimes Z_i+(Z_i\otimes Z_iL)K_{qq}.
$$
This is equivalent to 
$$
-vec(\Omega_i^{-1})'[Z_iL\otimes Z_i+(Z_i\otimes Z_iL)K_{qq}].
$$
From the 216 notes, I can simplify this using properties of $vec$ and the Kronecker product to get
$$
-vec(Z_i'\Omega_i^{-1}Z_iL).
$$
For the second part, I need $\partial/\partial L(tr(r_i'\Omega_i^{-1}r_i))$.
$$
\begin{align}
-\frac{\partial}{\partial L}tr(r_ir_i'\Omega_i^{-1})&=\Omega_i^{-1}r_ir_i'\Omega_i^{-1}\frac{\partial}{\partial L}(\sigma_0^2 I+Z_iLL'Z_i')\\
&=\Omega_i^{-1}r_ir_i'\Omega_i^{-1}(Z_iL\otimes Z_i+(Z_i\otimes Z_iL)K_{qq})\\
&=vec(\Omega_i^{-1}r_ir_i'\Omega_i^{-1})Z_iL\otimes Z_i\\
&=[(Z_iL)'\otimes Z_i']vec(\Omega_i^{-1}r_ir_i'\Omega_i^{-1})'\\
&=vec(Z_i'\Omega_i^{-1}r_ir_i'\Omega_i^{-1}Z_iL).
\end{align}
$$
Combining the two parts gives
$$
-vec(Z_i'\Omega_i^{-1}Z_iL)+vec(Z_i'\Omega_i^{-1}r_ir_i'\Omega_i^{-1}Z_iL).
$$
Reshaping this vector gives the desired answer of
$$
-Z_i'\Omega_i^{-1}Z_iL+Z_i'\Omega_i^{-1}r_ir_i'\Omega_i^{-1}Z_iL.
$$

To find the Fisher information, I need the Hessian.

1. $∇_β(X_i'Ω^{-1}(y-X_i\beta)=-X_i'\Omega_i^{-1}X_i$. The observed will then be this evaluated at the necessary MLEs, in this case $\hat\Omega_i^{-1}\equiv (\hat\sigma_0^2+Z_i\hat L\hat L'Z_i')$

2. Next is the Hessian wrt $\sigma_0^2:$
$$
\begin{align*}
&\nabla_{\sigma_0^2}\Big\{-\frac{1}{2}tr(\Omega^{-1}_i)+\frac{1}{2}r_i'\Omega_i^{-2}r_i)\Big\}\\
&=-\frac{1}{2}\Omega_i^{-2}+\nabla tr(r_ir_i'\Omega_i^{-2})\\
&=-\frac{1}{2}\Omega_i^{-2} + \frac{1}{2}r_ir_i'\Omega^{-1}+\frac{1}{2}r_ir_i'\Omega^{-1}\text{ by properties of derivatives of trace and the chain rule.}\\
&=-\frac{1}{2}\Omega_i^{-2}+r_ir_i'\Omega^{-1}
\end{align*}
$$
To find the observed information, I plug in $\hat\Omega_i^{-1}$.

3. The Hessian wrt $L$ is too icky for me.

## Q2. (20 pts) Objective and gradient evaluator for a single datum

We expand the code from HW2 to evaluate both objective and gradient. I provide my code for HW2 below as a starting point. You do _not_ have to use this code. If your come up faster code, that's even better. 

In [4]:
# define a type that holds an LMM datum
struct LmmObs{T <: AbstractFloat}
    # data
    y          :: Vector{T}
    X          :: Matrix{T}
    Z          :: Matrix{T}
    # arrays for holding gradient
    ∇β         :: Vector{T}
    ∇σ²        :: Vector{T}
    ∇Σ         :: Matrix{T}    
    # working arrays
    # TODO: whatever intermediate arrays you may want to pre-allocate
    yty        :: T
    xty        :: Vector{T}
    zty        :: Vector{T}
    zty2       :: Vector{T}
    storage_n  :: Vector{T}
    storage_p  :: Vector{T}
    storage_q  :: Vector{T}
    storage_q2 :: Vector{T}
    storage_q3 :: Vector{T}
    xtx        :: Matrix{T}
    ztx        :: Matrix{T}
    ztz        :: Matrix{T}
    ztz2        :: Matrix{T}
    storage_qq :: Matrix{T}
    storage_qq2 :: Matrix{T}
    storage_qq3 :: Matrix{T}
end

"""
    LmmObs(y::Vector, X::Matrix, Z::Matrix)

Create an LMM datum of type `LmmObs`.
"""
function LmmObs(
        y::Vector{T}, 
        X::Matrix{T}, 
        Z::Matrix{T}
    ) where T <: AbstractFloat
    n, p, q    = size(X, 1), size(X, 2), size(Z, 2)    
    ∇β         = Vector{T}(undef, p)
    ∇σ²        = Vector{T}(undef, 1)
    ∇Σ         = Matrix{T}(undef, q, q)    
    yty        = abs2(norm(y))
    xty        = transpose(X) * y
    zty        = transpose(Z) * y 
    zty2       = similar(zty)  
    storage_n  = Vector{T}(undef, n)
    storage_p  = Vector{T}(undef, p)
    storage_q  = Vector{T}(undef, q)
    storage_q2  = Vector{T}(undef, q)
    storage_q3  = Vector{T}(undef, q)
    xtx        = transpose(X) * X
    ztx        = transpose(Z) * X
    ztz        = transpose(Z) * Z
    ztz2        = similar(ztz)
    storage_qq = similar(ztz)
    storage_qq2 = similar(ztz)
    storage_qq3 = similar(ztz)
    LmmObs(y, X, Z, ∇β, ∇σ², ∇Σ, 
        yty, xty, zty, zty2, storage_n, storage_p, storage_q, storage_q2, storage_q3,
        xtx, ztx, ztz, ztz2, storage_qq, storage_qq2, storage_qq3)
end

"""
    logl!(obs::LmmObs, β, L, σ², needgrad=false)

Evaluate the log-likelihood of a single LMM datum at parameter values `β`, `L`, 
and `σ²`. If `needgrad==true`, then `obs.∇β`, `obs.∇Σ`, and `obs.σ² are filled 
with the corresponding gradient.
"""
function logl!(
        obs      :: LmmObs{T}, 
        β        :: Vector{T}, 
        L        :: Matrix{T}, 
        σ²       :: T,
        needgrad :: Bool = true
    ) where T <: AbstractFloat
    n, p, q = size(obs.X, 1), size(obs.X, 2), size(obs.Z, 2)
    ####################
    # Evaluate objective
    ####################    
    # form the q-by-q matrix: M = σ² * I + Lt Zt Z L
    copy!(obs.storage_qq, obs.ztz)
    BLAS.trmm!('L', 'L', 'T', 'N', T(1), L, obs.storage_qq) # O(q^3)
    BLAS.trmm!('R', 'L', 'N', 'N', T(1), L, obs.storage_qq) # O(q^3)
    @inbounds for j in 1:q
        obs.storage_qq[j, j] += σ²
    end
    # cholesky on M = σ² * I + Lt Zt Z L
    LAPACK.potrf!('U', obs.storage_qq) # O(q^3)
    # storage_q = (Mchol.U') \ (Lt * (Zt * res))
    BLAS.gemv!('N', T(-1), obs.ztx, β, T(1), copy!(obs.storage_q, obs.zty)) # O(pq)
    BLAS.trmv!('L', 'T', 'N', L, obs.storage_q)    # O(q^2)
    BLAS.trsv!('U', 'T', 'N', obs.storage_qq, obs.storage_q) # O(q^3)
    # l2 norm of residual vector
    copy!(obs.storage_p, obs.xty)
    rtr  = obs.yty +
        dot(β, BLAS.gemv!('N', T(1), obs.xtx, β, T(-2), obs.storage_p))
    # assemble pieces
    logl::T = n * log(2π) + (n - q) * log(σ²) # constant term
    @inbounds for j in 1:q
        logl += 2log(obs.storage_qq[j, j])
    end
    qf    = abs2(norm(obs.storage_q)) # quadratic form term
    logl += (rtr - qf) / σ² 
    logl /= -2
    ###################
    # Evaluate gradient
    ###################    
    if needgrad
        # TODO: fill ∇β, ∇L, ∇σ² by gradients
        
        # ∇β: X'Ω^{-1}r   
        # 1/σ²[X'r-X'ZLM⁻¹L'Z'r]
        # 1/σ²[X'r-X'Z * L * storage_q]
        copy!(obs.∇β, obs.xty) # get X'r
        BLAS.gemv!('N', T(-1), obs.xtx, β, T(1), obs.∇β)
        copy!(obs.storage_q2, obs.storage_q)
        BLAS.trsv!('U', 'N', 'N', obs.storage_qq, obs.storage_q2) # get M⁻¹Z'L'r
        BLAS.trmv!('L', 'N', 'N', L, obs.storage_q2) 
        BLAS.gemv!('T', T(-1), obs.ztx, obs.storage_q2, T(1), obs.∇β)
        obs.∇β ./= σ²
        
        # ∇σ²
        # second term: r'r - 2r'ZLM⁻¹L'Z'r + r'ZL M⁻¹ L'(Z'Z)L M⁻¹L'Z'r
        copy!(obs.storage_q3, obs.storage_q2)
        BLAS.gemm!('N', 'N', T(1), obs.ztz, obs.storage_q2, T(0), obs.storage_q3)
        obs.∇σ² .= rtr .- 2qf .+ dot(obs.storage_q2, obs.storage_q3)
        obs.∇σ² ./= σ²
        # first trace term: tr(ZLM⁻¹L'Z')=tr(LM⁻¹L'Z'Z)
        copy!(obs.storage_qq2, obs.ztz)
        BLAS.trmm!('L', 'L', 'T', 'N', T(1), L, obs.storage_qq2) # get L'Z'Z
        copy!(obs.storage_qq3, obs.storage_qq2) # use later for ∇Σ
        LAPACK.potrs!('U', obs.storage_qq, obs.storage_qq2) # get M⁻¹L'Z'Z
        BLAS.trmm!('L', 'L', 'N', 'N', T(1), L, obs.storage_qq2) # get LM⁻¹L'Z'Z
        @inbounds for i in 1:q
             obs.∇σ² .+= obs.storage_qq2[i, i]
        end
        obs.∇σ² .-= n 
        obs.∇σ² ./= (2 * σ²)
        
        # ∇Σ
        # get entire first term: Z'Ω⁻¹Z = Z'Z - Z'ZLM⁻¹L'Z'Z
        copy!(obs.ztz2, obs.ztz)
        BLAS.gemm!('N', 'N', T(-1), obs.ztz, obs.storage_qq2, T(1), obs.ztz2)
        obs.ztz2 ./= -σ²
        # get second term by breaking into (Z'Ω⁻¹r)(Z'Ω⁻¹r)'
        # Z'Ω⁻¹r = Z'(I-ZLM⁻¹L'Z')r = Z'r - Z'ZLM⁻¹L'Z'r
        copy!(obs.zty2, obs.zty)
        BLAS.gemv!('N', T(-1), obs.ztx, β, T(1), obs.zty2) # get Z'r = Z'y-Z'Xβ
        BLAS.gemm!('T', 'N', T(-1), obs.storage_qq2, obs.zty2, T(1), obs.zty2) # get Z'r - Z'ZLM⁻¹L'Z'r
        obs.zty2 ./= σ²
        BLAS.gemm!('N', 'T', T(1), obs.zty2, obs.zty2, T(0), obs.storage_qq3)
        broadcast!(+, obs.∇Σ, obs.ztz2, obs.storage_qq3)        
    end    
    ###################
    # Return
    ###################        
    return logl    
end

logl!

It is a good idea to test correctness and efficiency of the single datum objective/gradient evaluator here. First generate the same data set as in HW2.

In [5]:
Random.seed!(257)
# dimension
n, p, q = 2000, 5, 3
# predictors
X  = [ones(n) randn(n, p - 1)]
Z  = [ones(n) randn(n, q - 1)]
# parameter values
β  = [2.0; -1.0; rand(p - 2)]
σ² = 1.5
Σ  = fill(0.1, q, q) + 0.9I # compound symmetry 
L  = Matrix(cholesky(Symmetric(Σ)).L)
# generate y
y  = X * β + Z * rand(MvNormal(Σ)) + sqrt(σ²) * randn(n)

# form the LmmObs object
obs = LmmObs(y, X, Z);

### Correctness

In [6]:
@show logl = logl!(obs, β, L, σ², true)
@show obs.∇β
@show obs.∇σ²
@show obs.∇Σ

logl = logl!(obs, β, L, σ², true) = -3247.4568580638247
obs.∇β = [-1.6309843232717565, -77.52751558041872, -14.70237211601066, 6.978485518989568, -57.711823176822]
obs.∇σ² = [-4.820377758259156]
obs.∇Σ = [1.6423791649301933 1.8250240772238138 0.06127650043330851; 1.8250240772237973 0.11072391370504575 0.07213050869972704; 0.06127650043329193 0.07213050869968915 -1.0173748515304488]


3×3 Array{Float64,2}:
 1.64238    1.82502     0.0612765
 1.82502    0.110724    0.0721305
 0.0612765  0.0721305  -1.01737

You will lose all 20 points if following statement throws `AssertionError`.

In [7]:
@assert abs(logl - (-3247.4568580638247)) < 1e-8
@assert norm(obs.∇β - [-1.63098432327115, -77.52751558041871, -14.70237211601065, 
        6.978485518989568, -57.71182317682199]) < 1e-8
@assert abs(obs.∇σ²[1] - (-4.8203777582588145)) < 1e-8
@assert norm(obs.∇Σ - [1.6423791649290531 1.82502407722348 0.06127650043330721; 
        1.82502407722348 0.1107239137055005 0.07213050869971993; 
        0.06127650043330721 0.07213050869971993 -1.0173748515299939]) < 1e-8

### Efficiency

Benchmark for evaluating objective only. This is what we did in HW2.

In [8]:
@benchmark logl!($obs, $β, $L, $σ², false)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     710.421 ns (0.00% GC)
  median time:      768.571 ns (0.00% GC)
  mean time:        788.699 ns (0.00% GC)
  maximum time:     1.958 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     140

Benchmark for objective + gradient evaluation.

In [9]:
bm_objgrad = @benchmark logl!($obs, $β, $L, $σ², true)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.427 μs (0.00% GC)
  median time:      2.491 μs (0.00% GC)
  mean time:        2.583 μs (0.00% GC)
  maximum time:     15.211 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     9

My median runt time is 2.1μs. You will get full credit (10 pts) if the median run time is within 10μs.

In [10]:
#  The points you will get are
clamp(10 / (median(bm_objgrad).time / 1e3) * 10, 0, 10)

10.0

In [11]:
# # check for type stability
#@code_warntype logl!(obs, β, L, σ², true)

In [12]:
# using Profile

# Profile.clear()
# @profile for i in 1:10000; logl!(obs, β, L, σ², true); end
# Profile.print(format=:flat)

## Q3. LmmModel type

We create a `LmmModel` type to hold all data points and model parameters. Log-likelihood/gradient of a `LmmModel` object is simply the sum of log-likelihood/gradient of individual data points. 

In [13]:
# define a type that holds LMM model (data + parameters)
struct LmmModel{T <: AbstractFloat} <: MathProgBase.AbstractNLPEvaluator
    # data
    data :: Vector{LmmObs{T}}
    # parameters
    β    :: Vector{T}
    L    :: Matrix{T}
    σ²   :: Vector{T}    
    # arrays for holding gradient
    ∇β   :: Vector{T}
    ∇σ²  :: Vector{T}
    ∇L   :: Matrix{T}
    # TODO: add whatever intermediate arrays you may want to pre-allocate
    xty  :: Vector{T}
    ztr2 :: Vector{T}
    xtx  :: Matrix{T}
    ztz2 :: Matrix{T}
end

"""
    LmmModel(data::Vector{LmmObs})

Create an LMM model that contains data and parameters.
"""
function LmmModel(obsvec::Vector{LmmObs{T}}) where T <: AbstractFloat
    # dims
    p    = size(obsvec[1].X, 2)
    q    = size(obsvec[1].Z, 2)
    # parameters
    β    = Vector{T}(undef, p)
    L    = Matrix{T}(undef, q, q)
    σ²   = Vector{T}(undef, 1)    
    # gradients
    ∇β   = similar(β)    
    ∇σ²  = similar(σ²)
    ∇L   = similar(L)
    # intermediate arrays
    xty  = Vector{T}(undef, p)
    ztr2 = Vector{T}(undef, abs2(q))
    xtx  = Matrix{T}(undef, p, p)
    ztz2 = Matrix{T}(undef, abs2(q), abs2(q))
    LmmModel(obsvec, β, L, σ², ∇β, ∇σ², ∇L, xty, ztr2, xtx, ztz2)
end

"""
    logl!(m::LmmModel, needgrad=false)

Evaluate the log-likelihood of an LMM model at parameter values `m.β`, `m.L`, 
and `m.σ²`. If `needgrad==true`, then `m.∇β`, `m.∇Σ`, and `m.σ² are filled 
with the corresponding gradient.
"""
function logl!(m::LmmModel{T}, needgrad::Bool = false) where T <: AbstractFloat
    logl = zero(T)
    if needgrad
        fill!(m.∇β , 0)
        fill!(m.∇L , 0)
        fill!(m.∇σ², 0)        
    end
    @inbounds for i in 1:length(m.data)
        obs = m.data[i]
        logl += logl!(obs, m.β, m.L, m.σ²[1], needgrad)
        if needgrad
            BLAS.axpy!(T(1), obs.∇β, m.∇β)
            BLAS.axpy!(T(1), obs.∇Σ, m.∇L)
            m.∇σ²[1] += obs.∇σ²[1]
        end
    end
    # obtain gradient wrt L: m.∇L = m.∇L * L
    if needgrad
        BLAS.trmm!('R', 'L', 'N', 'N', T(1), m.L, m.∇L)
    end
    logl
end

logl!

## Q4. (20 pts) Test data

Let's generate a fake longitudinal data set to test our algorithm.

In [14]:
Random.seed!(257)

# dimension
m      = 1000 # number of individuals
ns     = rand(1500:2000, m) # numbers of observations per individual
p      = 5 # number of fixed effects, including intercept
q      = 3 # number of random effects, including intercept
obsvec = Vector{LmmObs{Float64}}(undef, m)
# true parameter values
βtrue  = [0.1; 6.5; -3.5; 1.0; 5]
σ²true = 1.5
σtrue  = sqrt(σ²true)
Σtrue  = Matrix(Diagonal([2.0; 1.2; 1.0]))
Ltrue  = Matrix(cholesky(Symmetric(Σtrue)).L)
# generate data
for i in 1:m
    # first column intercept, remaining entries iid std normal
    X = Matrix{Float64}(undef, ns[i], p)
    X[:, 1] .= 1
    @views Distributions.rand!(Normal(), X[:, 2:p])
    # first column intercept, remaining entries iid std normal
    Z = Matrix{Float64}(undef, ns[i], q)
    Z[:, 1] .= 1
    @views Distributions.rand!(Normal(), Z[:, 2:q])
    # generate y
    y = X * βtrue .+ Z * (Ltrue * randn(q)) .+ σtrue * randn(ns[i])
    # form a LmmObs instance
    obsvec[i] = LmmObs(y, X, Z)
end
# form a LmmModel instance
lmm = LmmModel(obsvec);

For later comparison with other software, we save the data into a text file `lmm_data.txt`. **Do not put this file in Git.** It takes 246.6MB storage.

In [15]:
(isfile("lmm_data.txt") && filesize("lmm_data.txt") == 246638945) || 
open("lmm_data.txt", "w") do io
    p = size(lmm.data[1].X, 2)
    q = size(lmm.data[1].Z, 2)
    # print header
    print(io, "ID,Y,")
    for j in 1:(p-1)
        print(io, "X" * string(j) * ",")
    end
    for j in 1:(q-1)
        print(io, "Z" * string(j) * (j < q-1 ? "," : "\n"))
    end
    # print data
    for i in eachindex(lmm.data)
        obs = lmm.data[i]
        for j in 1:length(obs.y)
            # id
            print(io, i, ",")
            # Y
            print(io, obs.y[j], ",")
            # X data
            for k in 2:p
                print(io, obs.X[j, k], ",")
            end
            # Z data
            for k in 2:q-1
                print(io, obs.Z[j, k], ",")
            end
            print(io, obs.Z[j, q], "\n")
        end
    end
end

true

### Correctness

Evaluate log-likelihood and gradient of whole data set at the true parameter values.

In [16]:
copy!(lmm.β, βtrue)
copy!(lmm.L, Ltrue)
lmm.σ²[1] = σ²true
@show obj = logl!(lmm, true)
@show lmm.∇β
@show lmm.∇σ²
@show lmm.∇L;

obj = logl!(lmm, true) = -2.854712648683302e6
lmm.∇β = [-8.693744360642468, 169.38364540290564, 1412.9462826018182, 583.9807190830948, 57.765860420243484]
lmm.∇σ² = [-371.8288642639251]
lmm.∇L = [20.197097749694237 -4.566719695057954 9.073934365189778; -5.895609775252421 -13.45609335371626 -18.889943728350204; 12.832481043334708 -20.692896580041015 -61.989536579199715]


Test correctness. You will loss all 20 points if following code throws `AssertError`.

In [17]:
@assert abs(obj - (-2.854712648683302e6)) < 1e-6
@assert norm(lmm.∇β - [-8.693744360651923, 169.38364540290684, 
        1412.9462826018173, 583.9807190830952, 57.76586042024306]) < 1e-6
@assert norm(lmm.∇L - [20.197097749713322 -4.566719695067792 9.073934365205824; 
        -5.895609775264991 -13.456093353707153 -18.889943728349024; 
        12.832481043357378 -20.69289658004 -61.98953657919662]) < 1e-6
@assert abs(lmm.∇σ²[1] - (-371.8288642639822)) < 1e-6

### Efficiency

Test efficiency.

In [18]:
bm_model = @benchmark logl!($lmm, true)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.494 ms (0.00% GC)
  median time:      2.577 ms (0.00% GC)
  mean time:        2.640 ms (0.00% GC)
  maximum time:     3.635 ms (0.00% GC)
  --------------
  samples:          1893
  evals/sample:     1

My median run time is 2.122ms. You will get full credit if your median run time is within 10ms. The points you will get are

In [19]:
clamp(10 / (median(bm_model).time / 1e6) * 10, 0, 10)

10.0

### Memory

You will lose 1 point for each 100 bytes memory allocation. So the points you will get is

In [20]:
clamp(10 - median(bm_model).memory / 100, 0, 10)

10.0

## Q5. (30 pts) Starting point

For numerical optimization, a good starting point is critical. Let's start $\boldsymbol{\beta}$ and $\sigma^2$ from the least sqaures solutions (ignoring intra-individual correlations)
\begin{eqnarray*}
\boldsymbol{\beta}^{(0)} &=& \left(\sum_i \mathbf{X}_i^T \mathbf{X}_i\right)^{-1} \left(\sum_i \mathbf{X}_i^T \mathbf{y}_i\right) \\
\sigma^{2(0)} &=& \frac{\sum_i \|\mathbf{r}_i^{(0)}\|_2^2}{\sum_i n_i} = \frac{\sum_i \|\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta}^{(0)}\|_2^2}{\sum_i n_i}.
\end{eqnarray*}
To get a reasonable starting point for $\boldsymbol{\Sigma} = \mathbf{L} \mathbf{L}^T$, we can minimize the least squares criterion (ignoring the noise variance component)
$$
    \text{minimize} \sum_i \| \mathbf{r}_i^{(0)} \mathbf{r}_i^{(0)T} - \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T \|_{\text{F}}^2.
$$
Derive the minimizer $\boldsymbol{\Sigma}^{(0)}$ (10 pts). 
We implement this start point strategy in the function `init_ls()`.

The minimizer is found by solving $\sum(Z_i^\top r_i\otimes Z^\top r_i)=[\sum_i(Z_i^\top Z_i\otimes Z_i^\top Z_i)]vec(\Sigma)$ for $\Sigma$.

This is derived as follows.

$\Vert r_ir_i^\top-X_i\beta\Vert^2_F$ is defined as $tr\big((r_ir_i^\top-X_i\beta)^\top(r_ir_i^\top-X_i\beta)\big)$. I'll expand this and then take the gradient with respect to $\Sigma$.
\begin{align}
&tr\big((r_ir_i^\top-X_i\beta)^\top(r_ir_i^\top-X_i\beta)\big)\\
=&tr(r_ir_i'r_ir_i'-r_ir_i'X\beta-\beta'X'r_ir_i'+\beta'X'X\beta)
\end{align}
Taking $\nabla\Sigma$ and equating to zero gives
\begin{align}
&\nabla tr(-r_ir_i'X\beta)-\nabla tr(\beta'X'r_ir_i')+\nabla tr(\beta'X'X\beta):=0.
\end{align}
Using line (116) of the matix cookbook to work with trace derivatives, I can simplify the expression to arrive at
$$
\begin{align}
\sum_i Z_i^\top Z_i\Sigma Z_i^\top Z_i-Z_i^\top r_ir_i^\top Z_i:=0.
\end{align}
$$
Using properties of $vec$ and the relation between outer products and the kronecker bilinear operator, I can rearrange this to arrive at the desired system of 
$$
\sum_i(Z_i^\top r_i\otimes Z^\top r_i)=[\sum_i(Z_i^\top Z_i\otimes Z_i^\top Z_i)]vec(\Sigma). 
$$

In [21]:
"""
    init_ls!(m::LmmModel)

Initialize parameters of a `LmmModel` object from the least squares estimate. 
`m.β`, `m.L`, and `m.σ²` are overwritten with the least squares estimates.
"""
function init_ls!(m::LmmModel{T}) where T <: AbstractFloat
    p, q = size(m.data[1].X, 2), size(m.data[1].Z, 2)
    # TODO: fill m.β, m.L, m.σ² by LS estimates
    
    # m.β
    @inbounds for i in 1:length(m.data)
        m.xtx .+= m.data[i].xtx
        m.xty .+= m.data[i].xty
    end
    LAPACK.potrf!('L', m.xtx)
    LAPACK.potrs!('L', m.xtx, m.xty)
    m.β .= m.xty
    
    # loop for σ² and Σ
    rsum = T(0)
    nsum = T(0)
    m.ztr2 .= T(0)
    m.ztz2 .= T(0)
    m.xtx .= T(0)
    m.xty .= T(0)
    obs.storage_qq .= T(0)
    @inbounds for i in 1:length(m.data)
        # get σ² start
        obs = m.data[i]
        rsum += obs.yty - 2dot(m.β,obs.xty) +
            dot(m.β, BLAS.gemv!('N',T(1), obs.xtx, m.β, T(0), obs.storage_p)) # for σ²
        nsum += size(obs.X, 1)
        
        # get Σ start      
        BLAS.gemv!('N', T(1), obs.ztx, m.β, T(0), obs.storage_q3) # get ztxβ
        BLAS.axpby!(T(1), obs.zty, T(-1), obs.storage_q3) # get Z'r = zty - ztxβ
        # now get Z'r ⊗ Z'r by using fact that u⊗v=vec(uv')
        BLAS.gemm!('N', 'T', T(1), obs.storage_q3, obs.storage_q3, T(0), obs.storage_qq) # form uv'
        m.ztr2 .+= vec(obs.storage_qq) # vec(uv')
        m.ztz2 .+= kron(obs.ztz, obs.ztz) # get Z'Z ⊗ Z'Z
    end
    m.σ² .= rsum / nsum
    LAPACK.potrf!('L', m.ztz2)
    LAPACK.potrs!('L', m.ztz2, m.ztr2)
    m.L .= reshape(m.ztr2, q, q)
    LAPACK.potrf!('L', m.L)
    m
end

init_ls!

In [22]:
init_ls!(lmm)
@show logl!(lmm)
@show lmm.β
@show lmm.σ²
@show lmm.L;

logl!(lmm) = -3.375070973605663e6
lmm.β = [0.08757868042872421, 6.49874064773743, -3.4970528646610326, 0.9971817478195717, 5.000992653697893]
lmm.σ² = [5.6672446612943475]
lmm.L = [1.4307910693942878 -0.0029959722919697404 0.018911797660621305; -0.0020939271680232518 1.08951604771318 -0.012600387416032014; 0.013217721346714464 -0.011539720315817313 0.9605029322102014]


### Correctness

Your start points should have a log-likelihood larger than -3.375071 (10 pts). The points you get are

In [23]:
# this is the points you get
(logl!(lmm) >  -3.375071e6) * 10

10

### Efficiency

The start point should be computed quickly. Otherwise there is no point using it as a starting point. You get full credit (10 pts) if the median run time is within 1ms.

In [24]:
bm_init = @benchmark init_ls!($lmm)

BenchmarkTools.Trial: 
  memory estimate:  796.97 KiB
  allocs estimate:  3002
  --------------
  minimum time:     587.978 μs (0.00% GC)
  median time:      636.246 μs (0.00% GC)
  mean time:        713.496 μs (10.76% GC)
  maximum time:     11.249 ms (92.30% GC)
  --------------
  samples:          6992
  evals/sample:     1

In [25]:
# this is the points you get
clamp(1 / (median(bm_init).time / 1e6) * 10, 0, 10)

10.0

## Q6. NLP via MathProgBase.jl

We define the NLP problem using the modelling tool MathProgBase.jl. Start-up code is given below. Modify if necessary to accomodate your own code.

In [26]:
"""
    fit!(m::LmmModel, solver=Ipopt.IpoptSolver(print_level=5))

Fit an `LmmModel` object by MLE using a nonlinear programming solver. Start point 
should be provided in `m.β`, `m.σ²`, `m.L`.
"""
function fit!(
        m::LmmModel,
        solver=Ipopt.IpoptSolver(print_level=5)
    )
    p    = size(m.data[1].X, 2)
    q    = size(m.data[1].Z, 2)
    npar = p + ((q * (q + 1)) >> 1) + 1
    optm = MathProgBase.NonlinearModel(solver)
    # set lower bounds and upper bounds of parameters
    # diagonal entries of Cholesky factor L should be >= 0
    lb   = fill(-Inf, npar)
    ub   = fill( Inf, npar)
    offset = p + 1
    for j in 1:q, i in j:q
        i == j && (lb[offset] = 0)
        offset += 1
    end
    # σ² should be >= 0
    lb[end] = 0
    MathProgBase.loadproblem!(optm, npar, 0, lb, ub, Float64[], Float64[], :Max, m)
    # starting point
    par0 = zeros(npar)
    modelpar_to_optimpar!(par0, m)
    MathProgBase.setwarmstart!(optm, par0)
    # optimize
    MathProgBase.optimize!(optm)
    optstat = MathProgBase.status(optm)
    optstat == :Optimal || @warn("Optimization unsuccesful; got $optstat")
    # update parameters and refresh gradient
    optimpar_to_modelpar!(m, MathProgBase.getsolution(optm))
    logl!(m, true)
    m
end

"""
    modelpar_to_optimpar!(par, m)

Translate model parameters in `m` to optimization variables in `par`.
"""
function modelpar_to_optimpar!(
        par :: Vector,
        m   :: LmmModel
    )
    p = size(m.data[1].X, 2)
    q = size(m.data[1].Z, 2)
    # β
    copyto!(par, m.β)
    # L
    offset = p + 1
    @inbounds for j in 1:q, i in j:q
        par[offset] = m.L[i, j]
        offset += 1
    end
    # σ²
    par[end] = m.σ²[1]
    par
end

"""
    optimpar_to_modelpar!(m, par)

Translate optimization variables in `par` to the model parameters in `m`.
"""
function optimpar_to_modelpar!(
        m   :: LmmModel, 
        par :: Vector
    )
    p = size(m.data[1].X, 2)
    q = size(m.data[1].Z, 2)
    # β
    copyto!(m.β, 1, par, 1, p)
    # L
    fill!(m.L, 0)
    offset = p + 1
    @inbounds for j in 1:q, i in j:q
        m.L[i, j] = par[offset]
        offset   += 1
    end
    # σ²
    m.σ²[1] = par[end]    
    m
end

function MathProgBase.initialize(
        m                  :: LmmModel, 
        requested_features :: Vector{Symbol}
    )
    for feat in requested_features
        if !(feat in [:Grad])
            error("Unsupported feature $feat")
        end
    end
end

MathProgBase.features_available(m::LmmModel) = [:Grad]

function MathProgBase.eval_f(
        m   :: LmmModel, 
        par :: Vector
    )
    optimpar_to_modelpar!(m, par)
    logl!(m, false) # don't need gradient here
end

function MathProgBase.eval_grad_f(
        m    :: LmmModel, 
        grad :: Vector, 
        par  :: Vector
    )
    p = size(m.data[1].X, 2)
    q = size(m.data[1].Z, 2)
    optimpar_to_modelpar!(m, par) 
    obj = logl!(m, true)
    # gradient wrt β
    copyto!(grad, m.∇β)
    # gradient wrt L
    offset = p + 1
    @inbounds for j in 1:q, i in j:q
        grad[offset] = m.∇L[i, j]
        offset += 1
    end
    # gradient with respect to σ²
    grad[end] = m.∇σ²[1]
    # return objective
    obj
end

MathProgBase.eval_g(m::LmmModel, g, par) = nothing
MathProgBase.jac_structure(m::LmmModel) = Int[], Int[]
MathProgBase.eval_jac_g(m::LmmModel, J, par) = nothing

## Q7. (20 pts) Test drive

Now we can run our NLP solver to compute the MLE. For grading purpose, we first use the `:LD_LBFGS` (limited-memory BFGS) algorithm in NLopt.jl here.

In [27]:
# initialize from least squares
init_ls!(lmm)
println("objective value at starting point: ", logl!(lmm)); println()

@time fit!(lmm, NLopt.NLoptSolver(algorithm=:LD_LBFGS, 
        ftol_rel = 1e-12, ftol_abs = 1e-12, 
        xtol_rel = 1e-12, xtol_abs = 1e-12, 
        maxeval=10000));

println("objective value at solution: ", logl!(lmm)); println()
println("solution values:")
@show lmm.β
@show lmm.σ²
@show lmm.L * transpose(lmm.L)
println("gradient @ solution:")
@show lmm.∇β
@show lmm.∇σ²
@show lmm.∇L
@show sqrt(abs2(norm(lmm.∇β)) + abs2(norm(lmm.∇σ²) + 
        abs2(norm(LowerTriangular(lmm.∇L)))))

objective value at starting point: -3.375070973605663e6

  0.753329 seconds (1.57 M allocations: 73.247 MiB, 1.86% gc time)
objective value at solution: -2.8547097852035086e6

solution values:
lmm.β = [0.08232306837570204, 6.500144672598971, -3.4987912684471874, 1.0004995460484036, 5.000049526624916]
lmm.σ² = [1.4990426771839431]
lmm.L * transpose(lmm.L) = [2.0568496310969113 -0.009911359051700368 0.01847432378988334; -0.009911359051700368 1.1822864462655216 -0.022704064306358605; 0.01847432378988334 -0.022704064306358605 0.9379064070639235]
gradient @ solution:
lmm.∇β = [-0.00018520309464842288, 0.0016361443523535968, 0.0034152542233520933, -0.0036719605909922848, -0.0026440576473198973]
lmm.∇σ² = [-0.0017264050714445034]
lmm.∇L = [0.007124035208945003 0.0017814144956104663 2.2090609804324766e-5; 0.0023684505349986975 -0.00022161502762673742 0.0012688481495313282; 2.604714324925945e-5 0.0014211770120903062 0.00017890322793720495]
sqrt(abs2(norm(lmm.∇β)) + abs2(norm(lmm.∇σ²) + abs2(nor

0.00616727811236522

### Correctness

You get 10 points if the following code does not throw `AssertError`.

In [28]:
# objective at solution should be close enough to the optimal
@assert logl!(lmm) > -2.85471e6
# gradient at solution should be small enough
@assert sqrt(abs2(norm(lmm.∇β)) + abs2(norm(lmm.∇σ²) + 
        abs2(norm(LowerTriangular(lmm.∇L))))) < 0.15

### Efficiency

My median run time is 132.5ms. You get 10 points if your median time is within 1s(=1000ms).

In [29]:
bm_bfgs = @benchmark fit!($lmm, $(NLopt.NLoptSolver(algorithm=:LD_LBFGS, 
        ftol_rel = 1e-12, ftol_abs = 1e-12, 
        xtol_rel = 1e-12, xtol_abs = 1e-12, 
        maxeval = 10000))) setup = (init_ls!(lmm))

BenchmarkTools.Trial: 
  memory estimate:  12.23 KiB
  allocs estimate:  267
  --------------
  minimum time:     166.055 ms (0.00% GC)
  median time:      186.029 ms (0.00% GC)
  mean time:        188.006 ms (0.00% GC)
  maximum time:     238.820 ms (0.00% GC)
  --------------
  samples:          27
  evals/sample:     1

In [30]:
# this is the points you get
clamp(1 / (median(bm_bfgs).time / 1e9) * 10, 0, 10)

10.0

## Q8. (10 pts) Gradient free vs gradient-based methods

Advantage of using a modelling tool such as MathProgBase.jl is that we can easily switch the backend solvers. For a research problem, we never know beforehand which solvers work better than the others. 

Try different solvers in the NLopt.jl and Ipopt.jl packages. Compare the results in terms run times (the shorter the better), objective values at solution (the larger the better), and gradients at solution (closer to 0 the better). Summarize what you find.

See this [page](https://nlopt.readthedocs.io/en/latest/NLopt_Algorithms/) for the descriptions of algorithms in NLopt.

Documentation for the Ipopt can be found [here](https://coin-or.github.io/Ipopt/). 

In [580]:
# vector of solvers to compare
solvers = [
    # NLopt: gradient-based algorithms
    NLopt.NLoptSolver(algorithm=:LD_LBFGS, 
        ftol_rel = 1e-12, ftol_abs = 1e-12,
        xtol_rel = 1e-12, xtol_abs = 1e-12,
        maxeval=10000),
    NLopt.NLoptSolver(algorithm=:LD_MMA, 
        ftol_rel = 1e-12, ftol_abs = 1e-12, 
        xtol_rel = 1e-12, xtol_abs = 1e-12, 
        maxeval=10000),
    # NLopt: gradient-free algorithms
    NLopt.NLoptSolver(algorithm=:LN_BOBYQA, 
        ftol_rel = 1e-12, ftol_abs = 1e-12, 
        xtol_rel = 1e-12, xtol_abs = 1e-12, 
        maxeval=10000),
    # Ipopt
    Ipopt.IpoptSolver(print_level=0)
]
# containers for results
runtime = zeros(length(solvers))
objvals = zeros(length(solvers))
gradnrm = zeros(length(solvers))

for (i, solver) in enumerate(solvers)
    bm = @benchmark fit!($lmm, $solver) setup = (init_ls!(lmm))
    runtime[i] = median(bm).time / 1e9
    objvals[i] = logl!(lmm, true)
    gradnrm[i] = sqrt(abs2(norm(lmm.∇β)) + abs2(norm(lmm.∇σ²) + 
        abs2(norm(LowerTriangular(lmm.∇L)))))
end

In [581]:
DataFrame(Runtime = runtime, Objective = objvals, Gradnorm = gradnrm)

,Runtime,Objective,Gradnorm
,Float64,Float64,Float64
1,0.154232,-2.85471e6,0.00616728
2,0.143592,-2.85471e6,0.000686134
3,0.0444582,-2.85603e6,35185.0
4,4.85844,-2.85471e6,1.45524e-5


## Q9. (10 pts) Compare with existing art

Let's compare our method with lme4 package in R and MixedModels.jl package in Julia. Both lme4 and MixedModels.jl are developed mainly by Doug Bates. Summarize what you find.

In [582]:
method  = ["257", "lme4", "MixedModels.jl"]
runtime = zeros(3)  # record the run times
loglike = zeros(3); # record the log-likelihood at MLE

### Your approach

In [562]:
bm_257 = @benchmark fit!($lmm, $(NLopt.NLoptSolver(algorithm=:LD_MMA, 
        ftol_rel = 1e-12, ftol_abs = 1e-12, 
        xtol_rel = 1e-12, xtol_abs = 1e-12, 
        maxeval=10000))) setup=(init_ls!(lmm))
runtime[1] = (median(bm_257).time) / 1e9
loglike[1] = logl!(lmm)

-2.8547097852035505e6

### lme4 

In [563]:
R"""
library(lme4)
library(readr)
library(magrittr)

testdata <- read_csv("lmm_data.txt")
"""

┌ Warning: RCall.jl: Loading required package: Matrix
└ @ RCall /Users/ianfrankenburg/.julia/packages/RCall/jOnwc/src/io.jl:160
┌ Warning: RCall.jl: Parsed with column specification:
│ cols(
│   ID = col_double(),
│   Y = col_double(),
│   X1 = col_double(),
│   X2 = col_double(),
│   X3 = col_double(),
│   X4 = col_double(),
│   Z1 = col_double(),
│   Z2 = col_double()
│ )
└ @ RCall /Users/ianfrankenburg/.julia/packages/RCall/jOnwc/src/io.jl:160


RObject{VecSxp}
# A tibble: 1,753,910 x 8
      ID      Y      X1      X2     X3      X4      Z1     Z2
   <dbl>  <dbl>   <dbl>   <dbl>  <dbl>   <dbl>   <dbl>  <dbl>
 1     1   2.30  2.16    1.71    1.95  -0.744   0.441  -0.155
 2     1  -7.82  0.591   0.0736 -1.39  -1.53   -0.284  -0.283
 3     1  -1.05 -0.666   0.223  -1.07   0.726  -0.701  -0.639
 4     1  -5.42 -0.364   0.181   1.61  -0.623   0.0332  0.905
 5     1  -4.23  0.880   1.91   -0.999 -0.371   1.03    0.581
 6     1  -2.72  0.341   0.162   0.603 -0.285   1.42   -0.831
 7     1   1.13 -0.0213 -0.500   0.178  0.747   0.638  -0.617
 8     1 -17.6  -1.76    0.899  -0.931 -0.0527  0.353  -1.38 
 9     1  -4.30 -0.207   0.248  -0.362 -0.0995 -0.587   0.266
10     1  20.9   2.24    0.523  -1.47   2.07    0.714   2.06 
# … with 1,753,900 more rows


In [564]:
R"""
rtime <- system.time(mmod <- 
  lmer(Y ~ X1 + X2 + X3 + X4 + (1 + Z1 + Z2 | ID), testdata, REML = FALSE))
"""

RObject{RealSxp}
   user  system elapsed 
 98.686   7.737 107.043 


In [565]:
R"""
rtime <- rtime["elapsed"]
summary(mmod)
rlogl <- logLik(mmod)
"""
runtime[2] = @rget rtime
loglike[2] = @rget rlogl;

### MixedModels.jl



In [566]:
testdata = CSV.File("lmm_data.txt", types = Dict(1=>String)) |> DataFrame!

,ID,Y,X1,X2,X3,X4,Z1
,String,Float64,Float64,Float64,Float64,Float64,Float64
1,1,2.29562,2.16439,1.71386,1.94501,-0.743783,0.441018
2,1,-7.82426,0.591156,0.0735725,-1.38799,-1.53388,-0.28432
3,1,-1.05162,-0.665533,0.222603,-1.06705,0.726237,-0.700676
4,1,-5.41677,-0.363982,0.181063,1.6054,-0.623457,0.0332452
5,1,-4.22634,0.880121,1.91223,-0.998628,-0.370723,1.02984
6,1,-2.72221,0.341423,0.161899,0.602739,-0.285159,1.4209
7,1,1.13149,-0.0212729,-0.500254,0.177658,0.747107,0.638381
8,1,-17.6405,-1.76199,0.898998,-0.931399,-0.0526501,0.352804
9,1,-4.30473,-0.206732,0.248303,-0.362355,-0.0994689,-0.58678


In [567]:
mj = fit(MixedModel, @formula(Y ~ X1 + X2 + X3 + X4 + (1 + Z1 + Z2 | ID)), testdata)
bm_mm = @benchmark fit(MixedModel, @formula(Y ~ X1 + X2 + X3 + X4 + (1 + Z1 + Z2 | ID)), testdata)
loglike[3] = loglikelihood(mj)
runtime[3] = median(bm_mm).time / 1e9

1.842507486

In [568]:
display(bm_mm)
mj

BenchmarkTools.Trial: 
  memory estimate:  920.42 MiB
  allocs estimate:  9433906
  --------------
  minimum time:     1.802 s (46.48% GC)
  median time:      1.843 s (50.34% GC)
  mean time:        1.877 s (47.84% GC)
  maximum time:     1.986 s (46.76% GC)
  --------------
  samples:          3
  evals/sample:     1

Linear mixed model fit by maximum likelihood
 Y ~ 1 + X1 + X2 + X3 + X4 + (1 + Z1 + Z2 | ID)
     logLik        -2 logLik          AIC             BIC       
 -2.85470979×10⁶  5.70941957×10⁶  5.70944357×10⁶   5.7095921×10⁶

Variance components:
            Column    Variance  Std.Dev.   Corr.
ID       (Intercept)  2.0568343 1.4341668
         Z1           1.1823375 1.0873534 -0.01
         Z2           0.9379257 0.9684656  0.01 -0.02
Residual              1.4990426 1.2243540
 Number of obs: 1753910; levels of grouping factors: 1000

  Fixed-effects parameters:
──────────────────────────────────────────────────────────
               Estimate    Std.Error      z value  P(>|z|)
──────────────────────────────────────────────────────────
(Intercept)   0.0823213  0.045357         1.81496   0.0695
X1            6.50014    0.000924999   7027.19      <1e-99
X2           -3.49879    0.000924895  -3782.91      <1e-99
X3            1.0005     0.000925797   1080.69      <1e-99
X4            5.0000

### Summary

In [569]:
DataFrame(method = method, runtime = runtime, logl = loglike)

,method,runtime,logl
,String,Float64,Float64
1,257,0.146077,-2.85471e6
2,lme4,107.043,-2.85471e6
3,MixedModels.jl,1.84251,-2.85471e6


## Q10. Be proud of yourself

Go to your resume/cv and claim you have experience performing analysis on complex longitudinal data sets with millions of records. And you beat current software by XXX fold. 